In [17]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [18]:
orders = pd.read_csv('/home/jupyter-l.fedoseeva-12/products_analytics/orders.csv', sep=',')

In [19]:
orders.head()

,id,order_date,ship_mode,customer_id,sales
0,100006,2014-09-07,Standard,DK-13375,377.970
1,100090,2014-07-08,Standard,EB-13705,699.192
2,100293,2014-03-14,Standard,NF-18475,91.056
3,100328,2014-01-28,Standard,JC-15340,3.928
4,100363,2014-04-08,Standard,JM-15655,21.376


In [20]:
orders.dtypes

id               int64
order_date      object
ship_mode       object
customer_id     object
sales          float64
dtype: object

In [21]:
orders['order_date'] = pd.to_datetime(orders.order_date)

In [23]:
Считаем дату первой покупки

Чтобы вычислить дату первой покупки каждого пользователя, сгруппируем данные по customer_id и найдем минимальное значение поля order_date. Результат сохраним в переменную first_orders:

SyntaxError: invalid syntax (<ipython-input-23-9393b3b9ab52>, line 1)

In [24]:
first_orders = orders.groupby('customer_id', as_index=False) \
        .agg({'order_date' : 'min'}) \
        .rename(columns={'order_date': 'first_date'})

In [25]:
first_orders.head()

,customer_id,first_date
0,AA-10315,2014-03-31
1,AA-10375,2014-04-21
2,AA-10480,2014-05-04
3,AA-10645,2014-06-22
4,AB-10015,2014-02-18


In [ ]:
# Второй способ
first_orders = orders.groupby('customer_id')['order_date'].agg({'first_order': 'min'})

In [ ]:
Строим когорты
Итак, когортами будем считать людей, сделавших первую покупку в тот или иной день. Метрики для отслеживания — количество заказов и выручка.

План такой: сначала добавим дату первой покупки пользователей в таблицу с заказами, затем сгруппируем по датам первой покупки и заказа и, наконец, посчитаем выручку и количество заказов каждой когорты.

Приступим. Добавим дату первой покупки с помощью метода merge() и сохраним получившийся датафрейм в переменную orders_merged:

In [30]:
orders_merge = orders.merge(first_orders, how='inner', on='customer_id')

In [31]:
orders_merge.head()

,id,order_date,ship_mode,customer_id,sales,first_date
0,100006,2014-09-07,Standard,DK-13375,377.970,2014-09-07
1,131884,2015-12-06,Same Day,DK-13375,594.002,2014-09-07
2,145065,2015-12-12,First,DK-13375,32.308,2014-09-07
3,133046,2017-07-27,Second,DK-13375,297.990,2014-09-07
4,165099,2017-12-11,First,DK-13375,1.392,2014-09-07


In [ ]:
В строках получившегося датафрейма всё еще покупки, но теперь в таблице появилась новая колонка: дата первой покупки пользователя.

Агрегируем по дате первой покупки и посчитаем нужные показатели:

In [36]:
orders_merge.groupby('first_date', as_index=False) \
            .agg({'sales' : ['sum', 'count']})

first_date      sales      
                      sum count
0   2014-01-03   1050.636     9
1   2014-01-04   1056.858     5
2   2014-01-05   1428.231     7
3   2014-01-06  14287.557    22
4   2014-01-07   1055.981     4
..         ...        ...   ...
441 2017-07-08   1058.108     1
442 2017-09-15   1038.260     1
443 2017-10-21    729.648     1
444 2017-10-23    863.880     1
445 2017-11-05   1205.584     2

[446 rows x 3 columns]

In [ ]:
Видим, что клиенты от 3 января 2014 года, всего сделали 9 заказов на $1050.6. Посмотрим, когда были эти заказы. Для этого добавим к группировке колонку order_date:

In [37]:
orders_merge.groupby(['first_date', 'order_date']) \
            .agg({'sales' : ['sum', 'count']})

sales      
                            sum count
first_date order_date                
2014-01-03 2014-01-03    16.448     1
           2014-11-12   153.112     1
           2015-04-18   209.550     1
           2015-11-24   383.610     1
           2016-05-15     7.764     1
...                         ...   ...
2017-09-15 2017-09-15  1038.260     1
2017-10-21 2017-10-21   729.648     1
2017-10-23 2017-10-23   863.880     1
2017-11-05 2017-11-05   796.036     1
           2017-11-18   409.548     1

[4621 rows x 2 columns]